# EP2 - Comparação entre 3 diferentes classificadores de Machine Learning aplicados sobre a base Adult da UCI
      
Autor do Estudo: Fernando Zolubas Preto - Nusp: 10694192

Classificadores e técnicas consideradas:
* Knn (K-nearest neighbor)
* Naive Bayes
* Redes Neurais
* Regressão Linear
* Regressão Não Linear
* Ridge
* Laço
* Regressão Logistica
* LDA  (Linear discriminant analysis)
* QDA
* Árvores de Classificação
* Bagging
* Random Forest
* Boosting
* Support Vector Machines SVMs

Esse estudo tem como limitação os seguintes aspectos-chave para a escolha dos classificadores a serem comparados:

* Tempo estimado para a realização do estudo: 2 dias (24h totais)
* Facilidade de interpretação dos resultados
* Facilidade de implementação dos classificadores

Dado esse cenário escolheu-se as seguintes técnicas com base nisso e nas aulas do curso:

* Regressão Logistica 
        Motivação: Implementação simples, Bom para classificacão binária, pouco sujeito a outliers.
* Random Forest
        Motivação: Implementação simples, Intendimento simples, pouco sujeito a outliers, recomendado pelo professor dados os bons resultados que costuma fornecer.
* Support Vector Machines
        Motivação: Pouco sujeito a outiliers, Não muito complexo.

# Data check

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd

In [16]:
adult = pd.read_csv(r"C:\Users\Fernando\Desktop\Poli Usp\4 semestre\Machine Learning\adult.csv",
        names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [17]:
adult.shape # nome_da_base.shape , shape == forma

(32561, 15)

Tem-se na base adult 32561 amostras, 14 features (=atributos) e 1 lablel (=rótulo). 

In [18]:
adult.head() # nome_da_base.head() , head() == cabeçalho {Pequena demonstração da base}

,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [19]:
nadult = adult.dropna() #Elimina-se observações com "missing data"

In [20]:
nadult.shape

(30162, 15)

In [21]:
adult.shape

(32561, 15)

In [22]:
Observacoes_com_missing_data = 32561 - 30162
print(Observacoes_com_missing_data) #numero absoluto de missing data
print((Observacoes_com_missing_data/32561)*100) #percentual de missing data

2399
7.367709836921471


In [23]:
#importantdo a base de teste
testAdult = pd.read_csv(r"C:\Users\Fernando\Desktop\Poli Usp\4 semestre\Machine Learning\adult.test.csv",
        names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [24]:
nTestAdult = testAdult.dropna()
print(testAdult.shape)
print(nTestAdult.shape)

(16282, 15)
(15060, 15)


Numero dos atributos
* Age________________0
* Workclass__________1
* fnlwgt_____________2
* Education__________3
* Education-Num______4
* Martial Stauts_____5
* Occupation_________6
* Relationship_______7
* Race_______________8
* Sex________________9
* Capital Gain______10
* Capital Loss______11
* Hours per week____12
* Country___________13

# Logistic Regression

* P(Y = 1|D) = (e^w)/(1 + e^w)
* w = somatória{i=1 to n}(beta_i * X_i)
* logit(P)

In [25]:
from sklearn.linear_model import LogisticRegression #Chamando a Regressao Logistica na biblioteca sklearn
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

Funções para regressão logistica

* Regressao Logistica com cross validation, predict[0,1], Toda a base

In [26]:
#Função Regressão Logistica com cross validation
def LR_CV(do_predict):
#Area de ajsustes de parametros da funcao:_______________________________________________________________
    pcv = 10 #Numero de divisoes da base de dados para cross validation
    
#A funcao:_______________________________________________________________________________________________
    from sklearn.linear_model import LogisticRegression #Chamando a Regressao Logistica na biblioteca sklearn
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn import preprocessing
    #Instanciando e organizando a base de dados
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,0:14]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,0:14]
    YtestAdult = numTestAdult.Target
    #Realizando a regressão Logistica e corss validation
    LR = LogisticRegression(random_state=0)
    scores_cv = cross_val_score(LR, Xadult, Yadult, cv=pcv, scoring='accuracy')
    
    #Observacao das features relevantes por meio dos pesos calculados, "betas"
    LR.fit(Xadult,Yadult)
    coefs = pd.Series(LR.coef_[0], index=Xadult.columns)
    coefs.sort_values(ascending = False)
    
    i=0
    average = 0
    while i<pcv :
        average = average + scores_cv[i]
        i = i + 1
    average = average/pcv
    
    if(do_predict == 1):
        YtestPred = LR.predict(XtestAdult)
        print(YtestPred)
    return average #Retorna a media dos resultados do cross validation

#COMO USAR: do_predict eh um booleano[0,1] que indica se
#uma previsao deve ser feita sobre a base de testes ou nao.

In [27]:
#Função para mostrar os pesos da regressao logistica, mostra "betas"
def betas():
    pcv = 10;
    from sklearn.linear_model import LogisticRegression #Chamando a Regressao Logistica na biblioteca sklearn
    
    #Instanciando e organizando a base de dados
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,0:14]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,0:14]
    YtestAdult = numTestAdult.Target
    #Realizando a regressão Logistica e corss validation
    LR = LogisticRegression(random_state=0)
    scores_cv = cross_val_score(LR, Xadult, Yadult, cv=pcv, scoring='accuracy')
    
    #Observacao das features relevantes por meio dos pesos calculados, "betas"
    LR.fit(Xadult,Yadult)
    coefs = pd.Series(LR.coef_[0], index=Xadult.columns)
    return coefs.sort_values(ascending = False)

In [28]:
#Função para mostrar os pesos da regressao logistica, mostra "betas"
#com selecao de atributos
def betas_select(Atributos):
    pcv = 10;
    from sklearn.linear_model import LogisticRegression #Chamando a Regressao Logistica na biblioteca sklearn
    
    #Instanciando e organizando a base de dados
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,Atributos]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,Atributos]
    YtestAdult = numTestAdult.Target
    
    #Observacao das features relevantes por meio dos pesos calculados, "betas"
    LR.fit(Xadult,Yadult)
    coefs = pd.Series(LR.coef_[0], index=Xadult.columns)
    return coefs.sort_values(ascending = False)


* Regressao Logistica com cross validation, predict[0,1], Selecao da base

In [29]:
#Função Regressão Logistica com cross validation e selecao de atributos via 
# vetor numerico 
def LR_cv_select(do_predict,Atributos):
    pcv = 10;
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn import preprocessing
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,Atributos]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,Atributos]
    YtestAdult = numTestAdult.Target
    #Realizando a regressão Logistica e corss validation
    LR = LogisticRegression(random_state=0)
    scores_cv = cross_val_score(LR, Xadult, Yadult, cv=pcv, scoring='accuracy')
    
    
    i=0
    average = 0
    while i<pcv :
        average = average + scores_cv[i]
        i = i + 1
    average = average/pcv
    
    if(do_predict == 1):
        LR.fit(Xadult,Yadult)
        YtestPred = LR.predict(XtestAdult)
        accuracy_pred = accuracy_score(YtestAdult,YtestPred)
        print(accuracy_pred)
    return average #Retorna a media dos resultados do cross validation
    

Execuções

In [116]:
LR_CV(1)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

[0 0 0 ... 0 1 0]


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8063469611893664

In [89]:
betas()

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

Education-Num     0.225347
Sex               0.082543
Capital Loss      0.029702
Capital Gain      0.027923
Age               0.018079
Hours per week    0.010725
fnlwgt           -0.000017
Occupation       -0.015136
Education        -0.027843
Country          -0.047948
Race             -0.127259
Workclass        -0.164646
Martial Status   -0.338541
Relationship     -0.368297
dtype: float64

Vemos que somente as features: Education-Num, Sex, Capital Loss, Capital Gain,
Age e Hours per week são de fato relevantes. Assim, chamemos betas_select(Atributos) para olhar somente para esses atributos descartando os demais de modo que o algoritimo aqui utilizado fez uma especie de data prep automático.

In [94]:
betas_select([4,9,11,10,0,12])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Sex               1.155933
Education-Num     0.332128
Age               0.043133
Hours per week    0.033003
Capital Loss      0.029212
Capital Gain      0.027598
dtype: float64

Curiosamente houve uma inversão com relação a relvancia das features dado que o atributo "sex" se mostrou muito mais relevante do que "Education-num" na aplicação do algoritimo em que se selecionou apenas os atributos relevantes.

Comparação: Data prep Intuitivo vs Betas da regressão Logística

No estudo anterior da base Adult (com o algoritimo KNN - EP1) numa estratégia
de bom senso e observações simples observou-se a otimização no KNN para
os seguintes atributos:
* Age, Education, Occupation, Relationship, Capital Gain, Capital Loss

Ao passo que para a regressão logística:
* Age, Education-Num, Hours per week, Sex, Capital Gain, Capital Loss

E de maneira divergente do esperado vemos que o atributo Relationship que apresenta a pior influencia na regressão logística se mostrou relevante para o KNN de acordo com esse data prep "intuitivo" do EP1 mostrando que algoritimos diferentes se baseiam em atributos eventualmente distintos e que a intuição não boa o bastante para guiar sozinha os metodos de aprendizado de máquina.

Comparacao entre KNN e regressao Logistica com parametros selecionados
* Metodo intuitivo vs Metodo estatistico


In [30]:
LR_cv_select(1,[4,9,11,10,0,12]) #Com parametros selecionados via RL

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

0.8175298804780876


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8173201139272305

In [101]:
LR_cv_select(0,[0,3,6,7,10,11]) #com parametros selecionados no data prep do EP1

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

0.7924546841628983

In [31]:
#Funcao KNN com cross validation
def knn_cv_select(KN,Atributos):
    pcv = 10;
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn import preprocessing
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,Atributos]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,Atributos]
    YtestAdult = numTestAdult.Target
    knn = KNeighborsClassifier(n_neighbors=KN)
    scores_cv = cross_val_score(knn, Xadult, Yadult, cv=pcv)
    
    i=0
    average = 0
    while i<pcv :
        average = average + scores_cv[i]
        i = i + 1
    average = average/pcv
    
    knn.fit(Xadult,Yadult)
    YtestPred = knn.predict(XtestAdult)
    accuracy_pred = accuracy_score(YtestAdult,YtestPred)
    print(accuracy_pred)
        
    return average

In [32]:
knn_cv_select(16,[0,3,6,7,10,11]) #com parametros selecionados no EP1 pra KNN

0.8280212483399735


0.8344279227975984

In [33]:
knn_cv_select(16,[4,9,11,10,0,12]) #com parametros selcionados via RegLogist

0.8164010624169987


0.8250121472076304

Obtivemos os seguintes resultados:
* RegLog(parametros via RegLog) = 0,817
* RegLog(parametros via intuicao) = 0,792
* KNN( parametros via RegLog) = 0,825
* KNN( parametros via intuicao) = 0,834



Esses resultados mostram que um algoritimo mais ligado ao conheciemnto teórico-estatístico como a Regressão Logística quando aplicado sem as devidas otimizações pode apresentar, ao contrario do que se possa imaginar a priori, um resultado pior em termos de acuracia do que um algoritimo intuitivo como o KNN realizado com as devidas otimizações. Em números isso se traduz pelos  81,7% de acuracia alcançados pela regressão logística frente aos 83,4% de acuracia alcançados pelo algoritimo KNN.

Lembrando que essas conclusões podem não ser gerais caso o problema em analise seja muito distindo do proposto pela base Adult.

**Tentativas de otimização da Regressão Logistica via seleção de parâmetros**

In [111]:
LR_cv_select(0,[4,9,0]) #somente os mais importantes

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

0.7913938957625677

In [112]:
LR_cv_select(0,[4,9,11,10,0,12,2]) #adicionando o melhor dos piores (fnlwgt)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

0.7887411450663944

In [114]:
LR_cv_select(0,[2,4,6,8,10,12]) #Alto grau de aleatoriedade em relacao ao problema


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

0.7793919789456536

**Base de testes:**

In [123]:
LR_cv_select(1,[4,9,11,10,0,12])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

0.8175298804780876


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8173201139272305

Dadas as limitações de tempo não foi possivel encontrar uma otimização melhor do que a proposta pelo proprio algoritimo cuja acuracia nesse caso foi de: 
* 81.73201139272305% para o Corss_Validation e de 
* 81.75298804780876% quando aplicado na base de testes.

Obs: Fica claro aqui a eficacia da estrategia de cross_validation haja vista a precisao da acuracia aplicada a base de testes que é a mesma daquela realcioanda ao cross validation.

***Suspeitas para o fato do KNN ter alcançado resultados melhores que a regressão logistica:***

O grau de linearidade do problema pode tal que de modo que a analise desse tipo de regressão fica prejudicada.

# Random Forest

In [129]:
#Funcao Random Forest com cross validation
def RF_cv_select_pred(do_predict,rs,Atributos):
    pcv = 10;
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn import preprocessing
    #Data base
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,Atributos]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,Atributos]
    YtestAdult = numTestAdult.Target
    #Processing Data
    RF = RandomForestClassifier(random_state=rs)
    scores_cv = cross_val_score(RF, Xadult, Yadult, cv=pcv)
    
    i=0
    average = 0
    while i<pcv :
        average = average + scores_cv[i]
        i = i + 1
    average = average/pcv
    
    if(do_predict == 1):
        RF.fit(Xadult,Yadult)
        YtestPred = RF.predict(XtestAdult)
        accuracy_pred = accuracy_score(YtestAdult,YtestPred)
        print("Teste")
        print(accuracy_pred)
    print("Cross_Validation")
    return average #Retorna a media dos resultados do cross validation

* Aplicações, levando em conta o hiper-parâmetro número de árvores

In [130]:
RF_cv_select_pred(1,1,[1,2,3,4,5,6,7,8,9,10,11,12,13]) #1 arvore

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8274900398406374
Cross_Validation


0.8347255179535971

In [131]:
RF_cv_select_pred(1,2,[1,2,3,4,5,6,7,8,9,10,11,12,13]) #2 arvores

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8271580345285524
Cross_Validation


0.8339962288266571

In [132]:
RF_cv_select_pred(1,10,[1,2,3,4,5,6,7,8,9,10,11,12,13]) #10 arvores

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8296812749003984
Cross_Validation


0.8347252431042806

In [133]:
RF_cv_select_pred(1,20,[1,2,3,4,5,6,7,8,9,10,11,12,13])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8248339973439575
Cross_Validation


0.8345267877986287

In [134]:
RF_cv_select_pred(1,50,[1,2,3,4,5,6,7,8,9,10,11,12,13])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8258300132802124
Cross_Validation


0.8341291626476292

In [135]:
RF_cv_select_pred(1,100,[1,2,3,4,5,6,7,8,9,10,11,12,13])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8288844621513944
Cross_Validation


0.8321725113184459

In [136]:
RF_cv_select_pred(1,500,[1,2,3,4,5,6,7,8,9,10,11,12,13])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8265604249667995
Cross_Validation


0.8320402482307017

Os melhores valores de acuraria foram para até 20 árvores. As medidas foram:
* 01) 0.8347255179535971
* 02) 0.8339962288266571
* 10) 0.8347252431042806
* 20) 0.8345267877986287


In [139]:
i=0
while i<21 :
    print(i,RF_cv_select_pred(1,i,[1,2,3,4,5,6,7,8,9,10,11,12,13]))
    i = i + 1

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8260292164674635
Cross_Validation
0 0.8334987383540462


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8274900398406374
Cross_Validation
1 0.8347255179535971


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

...

Teste
0.8249003984063745
Cross_Validation
18 0.8341292175241787


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8259628154050465
Cross_Validation
19 0.8328359932818827


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8248339973439575
Cross_Validation
20 0.8345267877986287


In [144]:
arr = np.array([0.8347255179535971,0.8339962288266571,0.8329685422307043,0.8336314633233639,0.8331009922850943,0.8338305562832211,0.8330681545952977,0.8343938319905859,0.8330680006639337,0.8347252431042806,0.8311115913310386,0.835256054931208,0.8337641112686788,0.833731405588787,0.8332335962343975,0.8344600570761023,0.8336314633671048,0.8341292175241787,0.8328359932818827,0.8345267877986287])
Index_Max_Accuracy = np.where(arr == np.amax(arr))
print('List of Indices of maximum element :', Index_Max_Accuracy[0])
# Get the maximum element from a Numpy array
print('Max accuracy is :', np.amax(arr))

List of Indices of maximum element : [11]
Max accuracy is : 0.835256054931208


**Regulagem do Hiper-parâmetro Número de Árvores para todos os atributos**

Dada a busca acima conclui-se que ao se considerar todas as features do data set temos que o número ótimo de árvores nesse caso é 11.
* Acuracia Cross Validation = 83.11115913310386%
* Acuracia Teste            = 82.80876494023904%

**Tentaiva de otimização, variando as features consideradas da base de dados**

Aqui se propões duas estratégias simples de otimização
* 1) Testar os resultados do Data preparation do EP1 
* 2) Testar os atributos indicados pela regressão Logística, isto é, usar os atributos cujos  "betas" são positivos.


TESTE 1) Resusltados do Data Prep do EP1

In [149]:
i=1
arr_res = []
while i<31 :
    arr_res.insert(i, RF_cv_select_pred(1,i,[0,3,6,7,10,11]))
    i = i + 1
arr = np.array(arr_res)
Index_Max_Accuracy = np.where(arr == np.amax(arr))
print('List of Indices of maximum element :', Index_Max_Accuracy[0])
# Get the maximum element from a Numpy array
print('Max accuracy is :', np.amax(arr))

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8262284196547145
Cross_Validation


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8276892430278885
Cross_Validation


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

...

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8262948207171315
Cross_Validation


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8270916334661355
Cross_Validation
List of Indices of maximum element : [19]
Max accuracy is : 0.8416224107069674


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


*Resultado* 1)
* List of Indices of maximum element : [19]
* Max accuracy is : 0.8416224107069674

TESTE 2) 
Features sugeridas pela Regressao Logistica

In [150]:
i=1
arr_res = []
while i<31 :
    rf = RF_cv_select_pred(1,i,[4,9,11,10,0,12])
    arr_res.insert(i, rf)
    print(i)
    i = i + 1
arr = np.array(arr_res)
Index_Max_Accuracy = np.where(arr == np.amax(arr))
print('List of Indices of maximum element :', Index_Max_Accuracy[0])
# Get the maximum element from a Numpy array
print('Max accuracy is :', np.amax(arr))

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.7926958831341302
Cross_Validation
1


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.7924966799468791
Cross_Validation
2


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

...

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.7967463479415671
Cross_Validation
29


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.796812749003984
Cross_Validation
30
List of Indices of maximum element : [19]
Max accuracy is : 0.8215634534964499


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


*Resultado* 2)
* List of Indices of maximum element : [19]
* Max accuracy is : 0.8215634534964499

In [152]:
RF_cv_select_pred(1,19,[0,3,6,7,10,11])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\f

Teste
0.8251660026560425
Cross_Validation


C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8396993226992284

**Resultados da Otimização**

A estraégia que mais funcionou foi a trataiva número 1 em que considerou-se o data Prep realizado no EP1 de onde se conclui que a escolha dos parâmetros:
Age, Education, Occupation, Relationship, Capital Gain, Capital Loss.
É a que mais otimiza a Random Forest de **19 árvores** com acuracia de 
* 84.16% Cross Validation
* 83.97% Teste

# SUPORT VECTOR MACHINE

Dado o alto consumo computacional desse algoritimo decidiu-se utilizar a versão
"Gaussian Kernel" (rbf) numa tentativa de reduzir a procura de hiper-parametros em relação a versão polinomial. 

In [12]:
#Funcao SVM - Polinomial gamma auto
def SVM_cv_select_pred_pol(do_predict,Atributos):
    pcv = 10;
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn import preprocessing
    #Data base
    numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
    numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
    Xadult = numAdult.iloc[:,Atributos]
    Yadult = numAdult.Target
    XtestAdult = numTestAdult.iloc[:,Atributos]
    YtestAdult = numTestAdult.Target
    #Processing Data
    SVM = SVC(kernel='rbf')
    scores_cv = cross_val_score(SVM, Xadult, Yadult, cv=pcv)
    
    i=0
    average = 0
    while i<pcv :
        average = average + scores_cv[i]
        i = i + 1
    average = average/pcv
    
    if(do_predict == 1):
        SVM.fit(Xadult,Yadult)
        YtestPred = SVM.predict(XtestAdult)
        accuracy_pred = accuracy_score(YtestAdult,YtestPred)
        print("Teste")
        print(accuracy_pred)
    print("Cross_Validation")
    return average #Retorna a media dos resultados do cross validation

**Execuções do SVM**

Serão adotas as duas estratégias anteriores como tentativas de Otimização.
Além disso será realizada uma execução aleatória de baixa intensidade computacional a fim de determinar um valor que tenda ao minimo da eficacia do algoritimo para que se tenha como métrica geral.

* 1) Execução Minima
* 2) Execução do Data Prep
* 3) Execução Com features sugeridas pelos pesos da Regressão Logística

* SVM com 1 variavel para testes, 3 min aproximadamente

In [16]:
SVM_cv_select_pred_pol(1,[0])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196:

Teste
0.7543160690571049
Cross_Validation


0.7510775258171767

* SVM com variaveis do Data Prep, 6 min aproximadamente

In [13]:
SVM_cv_select_pred_pol(1,[0,3,6,7,10,11])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196:

Teste
0.8047808764940239
Cross_Validation


0.8405280484618712

* SVM com variaveis do Data Prep, 10 min aproximadamente

In [15]:
SVM_cv_select_pred_pol(1,[4,9,11,10,0,12])

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\svm\base.py:196:

Teste
0.7924302788844622
Cross_Validation


0.8203036713150087

**Resultados SVM Classifier**
O melhor resultado para as iterações do SMV Classifier com Kernel Gaussiano foi para a seleção de atributos do Data Preparation como esperado alcançando as acuraricas de:

Teste:
* 80.47808764940239%

Cross_Validation
* 84.05280484618712%

Não Foram realizados mais testes devido a alto consumo computacional do algoritimo.

# Conclusões

Melhores Iterações:

*Teste*

* KNN:                     82.80 %
* Logistic Regression:     81.73 %
* Random Forest:           83.97 %
* Support Vector Machine:  80.48 %

*Cross Validation*

* KNN:                    83.44 %
* Logistic Regression:    81.75 %
* Random Forest:          84.16 %
* Support Vector Machine: 84.05 %


**Tempo**

* Os algoritimos Logistic Regression e Random Forest aprensentaram tempos similares de execução na ordem de poucos segundos enquanto o algoritimo SVM gastou dezenas de minutos para entregar o pior resultado de predição. 

* Isso pode estar relacionado a problemas de overfiting no SVM dado que o mesmo algoritimo que se ruim nos testes foi o melhor no processo de cross validation.

**Data Prep Automático ?**
Testou-se Utilizar nos métodos KNN, Random Forest e SVM as features de "peso positvo no calculo" no calulo dos "betas" da regressão logística. 
Resultados:

* KNN = Teste 0.816 Cross_Validation 0.825 pior_cv 0.727
* RandomForest = Teste 0.794 Cross_Validation  0.821 pior_cv 0.832
* SVM = Teste 0.79 Cross_Validation 0.82 pior_cv 0.751

Vemos que embora em nenhum desses três algoritimos o "data prep autolamtico" gerado pela regressão logística ganha de um data prep mais cuidadosamente elaborado como feito no EP1 para a base adult. Porém, essa estratatégia pode ser uma boa caso seja preciso elaborar um classificador muito rapidamente pois apresenta resultados de cross validation, e por tanto de teste, muito superiores do que os piores casos de cada algoritimo que seriam evitados usando essa proposta fornecida pela regressão logística.


**Acuraria**
    
O melhor algoritimo na base de teste, objetivo final de um algoritimo de Machine Learning antes de ser utilizado na prática da classificação, foi o Random Forest com incriveis 83,97 % de acuracia. Esse bom resultado se deve as propriedades do algoritimo que permite uma otima classificação para problemas do tipo, supervised Leraning proposto pela Base Adult, bem como o trabalho de Data Preparation que tornou possivel esse resultado garantindo mais (83.97 - 82.81 = 1.16 % de acurácia em relação ao caso não otimizado com as 14 features consideradas, um percentual alto quando a competição acima pela melhor eficacia esta na casa das unidades de percentual.